In [2]:
!pip -q install openai==1.61.0 tqdm huggingface_hub scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.6/460.6 kB 8.7 MB/s eta 0:00:00


In [3]:
import os, json, random, time
from pathlib import Path

PROJECT_DIR = Path("/content/ubp_project_clean")
DATA_DIR = PROJECT_DIR / "data"
OUT_DIR = PROJECT_DIR / "outputs"
DATA_DIR.mkdir(parents=True, exist_ok=True)
OUT_DIR.mkdir(parents=True, exist_ok=True)

# 读取 GEMINI_API_KEY：优先环境变量，其次 Colab Secrets
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")
if not GEMINI_API_KEY:
    try:
        from google.colab import userdata
        GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")
    except Exception:
        GEMINI_API_KEY = None

assert GEMINI_API_KEY, "❌ GEMINI_API_KEY 没读到：请在 Colab Secrets 里设置 GEMINI_API_KEY 并重启 runtime"

print("✅ Project:", PROJECT_DIR)
print("✅ GEMINI_API_KEY loaded:", bool(GEMINI_API_KEY))

✅ Project: /content/ubp_project_clean
✅ GEMINI_API_KEY loaded: True


In [4]:
!git clone --depth 1 https://github.com/LivXue/SoMe.git /content/SoMe

Cloning into '/content/SoMe'...
remote: Enumerating objects: 205, done.
remote: Counting objects: 100% (205/205), done.
remote: Compressing objects: 100% (144/144), done.
remote: Total 205 (delta 51), reused 193 (delta 48), pack-reused 0 (from 0)
Receiving objects: 100% (205/205), 9.96 MiB | 9.59 MiB/s, done.
Resolving deltas: 100% (51/51), done.


In [5]:
gt_path = "/content/SoMe/datasets/user_behavior_prediction/ground_truth.json"
print("exists:", os.path.exists(gt_path))

exists: True


In [6]:
from huggingface_hub import hf_hub_download

post_path = hf_hub_download(
    repo_id="LivXue/SoMe",
    repo_type="dataset",
    filename="database/post_data/all_posts.json"
)

print("all_posts path:", post_path)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


database/post_data/all_posts.json:   0%|          | 0.00/303M [00:00<?, ?B/s]

all_posts path: /root/.cache/huggingface/hub/datasets--LivXue--SoMe/snapshots/33c8a1c48b01edd8c7be7fab0a29f6e1ad2181e0/database/post_data/all_posts.json


In [8]:
import json, os

GT_PATH = "/content/SoMe/datasets/user_behavior_prediction/ground_truth.json"
POST_PATH = "/root/.cache/huggingface/hub/datasets--LivXue--SoMe/snapshots"

# 找到all_posts.json真实路径
def find_all_posts(base):
    for root, dirs, files in os.walk(base):
        for f in files:
            if f == "all_posts.json":
                return os.path.join(root, f)

POST_PATH = find_all_posts(POST_PATH)

print("GT:", GT_PATH)
print("POST:", POST_PATH)

gt = json.load(open(GT_PATH,"r",encoding="utf-8"))
posts = json.load(open(POST_PATH,"r",encoding="utf-8"))

print("users:",len(gt["like"]))
print("posts:",len(posts))

GT: /content/SoMe/datasets/user_behavior_prediction/ground_truth.json
POST: /root/.cache/huggingface/hub/datasets--LivXue--SoMe/snapshots/33c8a1c48b01edd8c7be7fab0a29f6e1ad2181e0/database/post_data/all_posts.json
users: 500
posts: 833609


In [9]:
samples = []

for uid, d in gt["like"].items():

    for idx, item in enumerate(d):

        wid = item["weibo_id"]
        label = item["label"]

        if wid in posts:

            samples.append(
                (
                    uid,
                    idx,
                    wid,
                    label,
                    posts[wid]
                )
            )

samples200 = samples[:200]

print("samples:",len(samples200))

samples: 200


In [10]:
import os
from openai import OpenAI

# 读取 GEMINI_API_KEY：优先环境变量，其次 Colab Secrets
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")
if not GEMINI_API_KEY:
    try:
        from google.colab import userdata
        GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")
    except Exception:
        GEMINI_API_KEY = None

assert GEMINI_API_KEY, "❌ GEMINI_API_KEY 没读到：请在 Colab Secrets 里设置 GEMINI_API_KEY 并重启 runtime"

BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
MODEL_NAME = "gemini-3-flash-preview"

client = OpenAI(api_key=GEMINI_API_KEY, base_url=BASE_URL)

print("✅ GEMINI key loaded:", bool(GEMINI_API_KEY))
print("✅ model:", MODEL_NAME)

✅ GEMINI key loaded: True
✅ model: gemini-3-flash-preview


In [11]:
import signal
import time
class TimeoutException(Exception):
    pass

def _timeout_handler(signum, frame):
    raise TimeoutException()

signal.signal(signal.SIGALRM, _timeout_handler)

def ask_llm(prompt, timeout_sec=45, max_retries=3):
    last_err = None

    for attempt in range(max_retries + 1):
        try:
            r = client.chat.completions.create(
                model=MODEL_NAME,
                messages=[{"role": "user", "content": prompt}],
                temperature=0,
                timeout=timeout_sec
            )

            return r.choices[0].message.content

        except Exception as e:
            err = str(e)
            last_err = err

            retryable = any(k in err for k in [
                "Timeout", "ReadTimeout", "ConnectTimeout",
                "429", "Rate limit", "503", "502", "500"
            ])

            if retryable and attempt < max_retries:
                sleep_s = min(2 ** attempt, 8)
                print("Retry after error:", err)
                time.sleep(sleep_s)
                continue

            if "Timeout" in err:
                return "TIMEOUT"

            return "ERROR: " + err

    return "ERROR: " + str(last_err)

def parse_yesno(text: str):
    t = (text or "").strip()
    if "是" in t and "否" not in t:
        return "是"
    if "否" in t and "是" not in t:
        return "否"
    # 若两者都出现：取最后出现的那个
    last_is = t.rfind("是")
    last_no = t.rfind("否")
    if last_is == -1 and last_no == -1:
        return None
    return "是" if last_is > last_no else "否"

def get_post_text(post: dict) -> str:
    return str(post.get("内容", "")).strip()

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def norm_label(x):
    s = str(x).strip()
    s = s.strip('"').strip("'").strip()
    return s  # "是"/"否"

def build_user_history(uid: str, target_wid: str, k=5):
    """
    从 ground_truth like 中取该用户 label=是 的历史微博（排除 target）
    返回 list[str]（微博文本）
    """
    items = gt["like"].get(uid, [])
    liked_wids = []
    for it in items:
        wid = str(it.get("weibo_id"))
        if wid == target_wid:
            continue
        if norm_label(it.get("label")) == "是" and wid in posts:
            liked_wids.append(wid)
    liked_wids = liked_wids[:k]
    return [get_post_text(posts[w]) for w in liked_wids]

def retrieve_similar_from_history(target_text: str, history_texts: list[str], topm=3):
    if not history_texts:
        return []
    corpus = history_texts + [target_text]
    vec = TfidfVectorizer(max_features=2000)
    X = vec.fit_transform(corpus)
    sims = cosine_similarity(X[-1], X[:-1]).flatten()
    idxs = sims.argsort()[::-1][:topm]
    return [history_texts[i] for i in idxs]

def prompt_baseline(post_text: str) -> str:
    return f"""你是社交媒体行为预测助手。
任务：判断“某用户”是否会对这条微博点赞。

微博内容：
{post_text}

只输出一个字：是 或 否
"""

def prompt_history(post_text: str, history_texts: list[str]) -> str:
    history_block = "\n".join([f"- {t}" for t in history_texts]) if history_texts else "(无)"
    return f"""你是社交媒体行为预测助手。
你会看到该用户“曾经点赞过的微博内容”（少量），请基于历史偏好预测他是否会点赞目标微博。

用户历史点赞内容：
{history_block}

目标微博内容：
{post_text}

只输出一个字：是 或 否
"""

def prompt_retrieval(post_text: str, retrieved_texts: list[str]) -> str:
    retrieved_block = "\n".join([f"- {t}" for t in retrieved_texts]) if retrieved_texts else "(无)"
    return f"""你是社交媒体行为预测助手。
已从该用户历史点赞内容中检索出与目标微博最相近的几条作为证据，请基于证据判断他是否会点赞目标微博。

检索到的相似历史点赞微博：
{retrieved_block}

目标微博内容：
{post_text}

只输出一个字：是 或 否
"""

In [13]:
import json
from pathlib import Path
from tqdm import tqdm

OUT_DIR = Path("/content/ubp_project_clean/outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

def run_method(method_name: str, samples, out_json_path: Path,
               k_history=5, topm=3, timeout_sec=60, save_every=10):
    """
    保存格式：
    {
      "点赞": {
        uid: { idx: "是/否/TIMEOUT/ERROR:xxx/UNPARSED" }
      }
    }
    """
    results = {"点赞": {}}

    # 断点续跑
    if out_json_path.exists():
        try:
            results = json.load(open(out_json_path, "r", encoding="utf-8"))
        except Exception:
            pass

    for uid, idx, wid, label, post in tqdm(samples, desc=method_name):
        uid = str(uid); idx = str(idx); wid = str(wid)
        results["点赞"].setdefault(uid, {})

        if idx in results["点赞"][uid]:
            continue

        post_text = get_post_text(post)

        if method_name == "baseline":
            prompt = prompt_baseline(post_text)

        elif method_name == "history":
            hist = build_user_history(uid, wid, k=k_history)
            prompt = prompt_history(post_text, hist)

        elif method_name == "retrieval_text":
            hist = build_user_history(uid, wid, k=k_history)
            retrieved = retrieve_similar_from_history(post_text, hist, topm=topm)
            prompt = prompt_retrieval(post_text, retrieved)

        else:
            raise ValueError("unknown method")

        raw = ask_llm(prompt, timeout_sec=timeout_sec)
        yn = parse_yesno(raw)

        if yn is None:
            if raw.startswith("TIMEOUT") or raw.startswith("ERROR"):
                pred = raw
            else:
                pred = "UNPARSED"
        else:
            pred = yn

        results["点赞"][uid][idx] = pred

        # 定期保存
        count_preds = sum(len(v) for v in results["点赞"].values())
        if count_preds % save_every == 0:
            with open(out_json_path, "w", encoding="utf-8") as f:
                json.dump(results, f, ensure_ascii=False, indent=2)

    # 最后强制保存
    with open(out_json_path, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

    return results

def evaluate(results, samples):
    gold = {(str(uid), str(idx)): norm_label(label) for uid, idx, wid, label, post in samples}

    total = 0
    parsed = 0
    correct = 0
    unparsed = 0
    error_like = 0

    for (uid, idx), lab in gold.items():
        total += 1
        pred = results.get("点赞", {}).get(uid, {}).get(idx, None)

        if pred in ["是", "否"]:
            parsed += 1
            if pred == lab:
                correct += 1
        else:
            unparsed += 1
            if isinstance(pred, str) and (pred.startswith("ERROR") or pred.startswith("TIMEOUT")):
                error_like += 1

    tcr = parsed / total if total else 0
    acc_known = correct / parsed if parsed else 0
    return {"Total": total, "TCR": tcr, "ACC_known": acc_known, "Unparsed": unparsed, "Error_like": error_like}

In [16]:
test_samples = samples200[:5]

for m in ["baseline", "history", "retrieval_text"]:
    outp = OUT_DIR / f"{MODEL_NAME}_{m}_n5.json"
    print("\n=== Running", m, "on 5 samples ===")
    res = run_method(m, test_samples, outp, k_history=5, topm=3, timeout_sec=60, save_every=2)
    print("Saved:", outp)
    print("Eval:", evaluate(res, test_samples))


=== Running baseline on 5 samples ===


baseline: 100%|██████████| 5/5 [00:24<00:00,  4.98s/it]


Saved: /content/ubp_project_clean/outputs/gemini-3-flash-preview_baseline_n5.json
Eval: {'Total': 5, 'TCR': 1.0, 'ACC_known': 0.6, 'Unparsed': 0, 'Error_like': 0}

=== Running history on 5 samples ===


history: 100%|██████████| 5/5 [00:19<00:00,  3.92s/it]


Saved: /content/ubp_project_clean/outputs/gemini-3-flash-preview_history_n5.json
Eval: {'Total': 5, 'TCR': 1.0, 'ACC_known': 0.8, 'Unparsed': 0, 'Error_like': 0}

=== Running retrieval_text on 5 samples ===


retrieval_text: 100%|██████████| 5/5 [00:22<00:00,  4.46s/it]

Saved: /content/ubp_project_clean/outputs/gemini-3-flash-preview_retrieval_text_n5.json
Eval: {'Total': 5, 'TCR': 1.0, 'ACC_known': 0.4, 'Unparsed': 0, 'Error_like': 0}


In [19]:
final_reports = {}

for m in ["baseline", "history", "retrieval_text"]:
    outp = OUT_DIR / f"{MODEL_NAME}_{m}_n200.json"
    print("\n==============================")
    print("✅ Running", m, "on 200 samples")
    res = run_method(m, samples200, outp, k_history=5, topm=3, timeout_sec=60, save_every=10)
    rep = evaluate(res, samples200)
    final_reports[m] = rep
    print("Saved:", outp)
    print("Report:", rep)

print("\n✅ All done. Summary:")
print(json.dumps(final_reports, ensure_ascii=False, indent=2))


✅ Running baseline on 200 samples


baseline: 100%|██████████| 200/200 [00:00<00:00, 280087.08it/s]


Saved: /content/ubp_project_clean/outputs/gemini-3-flash-preview_baseline_n200.json
Report: {'Total': 200, 'TCR': 0.995, 'ACC_known': 0.45226130653266333, 'Unparsed': 1, 'Error_like': 1}

✅ Running history on 200 samples


history:   1%|          | 2/200 [00:09<15:18,  4.64s/it]


KeyboardInterrupt: 

In [18]:
res = run_method("baseline", samples200, outp, timeout_sec=45)

baseline: 100%|██████████| 200/200 [07:16<00:00,  2.18s/it]


In [20]:
outp = OUT_DIR / f"{MODEL_NAME}_history_n200.json"

res = run_method(
    "history",
    samples200,
    outp,
    k_history=5,
    timeout_sec=45
)

print(evaluate(res, samples200))

history: 100%|██████████| 200/200 [13:46<00:00,  4.13s/it]

{'Total': 200, 'TCR': 1.0, 'ACC_known': 0.4, 'Unparsed': 0, 'Error_like': 0}


In [23]:
outp = OUT_DIR / f"{MODEL_NAME}_retrieval_text_n200.json"

res = run_method(
    "retrieval_text",
    samples200,
    outp,
    k_history=5,
    topm=3,
    timeout_sec=45
)

print(evaluate(res, samples200))

retrieval_text: 100%|██████████| 200/200 [12:46<00:00,  3.83s/it]

{'Total': 200, 'TCR': 1.0, 'ACC_known': 0.32, 'Unparsed': 0, 'Error_like': 0}


In [26]:
import json
import os

# ========= 1. 正确的 ground truth 路径 =========
gt_path = "/content/SoMe/datasets/user_behavior_prediction/ground_truth.json"

# ========= 2. 结果文件路径 =========
base_dir = "/content/ubp_project_clean/outputs"

baseline_path   = os.path.join(base_dir, "gemini-3-flash-preview_baseline_n200.json")
history_path    = os.path.join(base_dir, "gemini-3-flash-preview_history_n200.json")
retrieval_path  = os.path.join(base_dir, "gemini-3-flash-preview_retrieval_text_n200.json")

# ========= 3. 读取数据 =========
gt = json.load(open(gt_path))

baseline_pred = json.load(open(baseline_path))
history_pred = json.load(open(history_path))
retrieval_pred = json.load(open(retrieval_path))

# ========= 4. 构建 GT map =========
gt_map = {}

for uid, arr in gt["like"].items():
    for idx, item in enumerate(arr):
        gt_map[(uid, str(idx))] = item["label"]

# ========= 5. 计算 ACC =========
def calc_acc(pred):

    correct = 0
    total = 0

    for uid, d in pred["点赞"].items():
        for idx, ans in d.items():

            key = (uid, idx)

            if key not in gt_map:
                continue

            gt_label = gt_map[key]

            if ans in ["是", "否"]:
                total += 1
                if ans == gt_label:
                    correct += 1

    if total == 0:
        return 0

    return correct / total


acc_baseline = calc_acc(baseline_pred)
acc_history = calc_acc(history_pred)
acc_retrieval = calc_acc(retrieval_pred)

# ========= 6. 打印结果 =========
print("========== FINAL RESULT ==========")
print(f"Baseline ACC      : {acc_baseline:.5f}")
print(f"History ACC       : {acc_history:.5f}")
print(f"Retrieval ACC     : {acc_retrieval:.5f}")

========== FINAL RESULT ==========
Baseline ACC      : 0.45226
History ACC       : 0.40000
Retrieval ACC     : 0.32000
